In [1]:
from yolox.tracker.byte_tracker import BYTETracker
from yolox.utils.visualize import plot_tracking
# from bytetrack import BYTETracker
from ultralytics.utils import LOGGER
from ultralytics import YOLO
import cv2

import numpy as np
import torch

# yolo 로그 메시지 비활성화
LOGGER.disabled = True

In [2]:
# YOLOv8 모델 로드 (사전 학습된 COCO 데이터셋 사용)
model = YOLO('yolov8n.pt')  # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt' 중 선택 가능

In [3]:
# ByteTrack 설정
tracker_args = {
    'track_thresh': 0.3,
    'high_thresh': 0.6, 
    'match_thresh': 0.8,
    'track_buffer': 30,
    'mot20': False
}

# 딕셔너리를 Namespace로 변환
from argparse import Namespace
tracker_args = Namespace(**tracker_args)

# # BYTETracker 클래스 내부에서 딕셔너리 처리
# class BYTETracker:
#     def __init__(self, args, frame_rate=30):
#         # 딕셔너리로 처리
#         self.det_thresh = args['track_thresh'] + 0.1
#         self.buffer_size = int(frame_rate / 30.0 * args['track_buffer'])
#         self.max_time_lost = self.buffer_size
#         # 기타 초기화 코드

tracker = BYTETracker(tracker_args)

In [46]:
# 2. 로컬 영상 파일 열기
video_path = '../../../Data/video/1-1_601-C09.mp4'
url = 'rtsp://210.99.70.120:1935/live/cctv002.stream'
cap = cv2.VideoCapture(url)

In [47]:
# 3. 프레임별 처리
object_tracks = {}  # 객체 위치 저장용 딕셔너리
count_num = 0
TARGET_CLASS = [2, 5, 7]

frame_count = 0
skip_frames = 5  # 5프레임마다 한 번씩 처리

# 첫 번째 프레임에서 프레임 크기 초기화
ret, frame = cap.read()
if not ret:  # 첫 번째 프레임 읽기 실패 시 프로그램 종료
    print("비디오 파일을 읽을 수 없습니다.")
    cap.release()
    exit()

frame_height, frame_width = frame.shape[:2]
print(frame_height, frame_width)
mid_x = frame_width // 2  # 화면 중간 x좌표 계산
mid_y = frame_height // 2  # 화면 중간 y좌표 계산

print("영상시작")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % skip_frames != 0:  # 프레임 스킵
        continue
    if frame_count % 1000 == 0:  # 1000프레임마다 스트림 재연결
        print("스트림 초기화")
        cap.release()
        cap = cv2.VideoCapture(url)

    # YOLOv8 객체 탐지
    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes:
            # print("box", box)
            x1, y1, x2, y2 = box.xyxy[0].tolist() #map(float, box.xyxy[0].tolist())  # 좌표를 계산하는 부분
            confidence = float(box.conf[0])  # tensor -> float 변환  # 신뢰도
            class_id = int(box.cls[0])  # 클래스id
            if confidence > 0.5 and class_id in TARGET_CLASS:  # 신뢰도 필터링
                detections.append([x1, y1, x2, y2, confidence, class_id])  # [x1, y1, x2, y2, confidence, class_id]
    
    # ByteTrack 추적
    img_info = (frame_height, frame_width)  # 이미지 정보
    img_size = (frame_width, frame_height)  # 이미지 크기

    if len(detections) == 0:
        detection_array = np.empty((0, 5), dtype=float)  # 빈 배열 생성
    else:
        detection_array = np.array(detections, dtype=float)  # 넘파이 버전 이슈로 dtype수정
    # print(f"detections: {detections}")
    detection_tensor = torch.from_numpy(detection_array).type(torch.float32)  # numpy 배열을 PyTorch Tensor로 변환
    online_targets = tracker.update(detection_tensor, img_info, img_size)
    
    # 추적 결과 시각화
    for target in online_targets:
        # print("target.tlwh:", target.tlwh)
        x1, y1, w, h = target.tlwh  # 바운딩 박스 좌표 및 크기
        x1, y1, x2, y2 = int(x1), int(y1), int(x1 + w), int(y1 + h)  # 박스 좌표 변환

        # YOLO 모델의 입력 이미지 크기 (전처리 시 사용한 크기)
        input_width, input_height = 1080, 720  # 모델 입력 크기를 가져옴
        # 이미지 크기 비율 계산
        scale_x = frame_width / input_width  # 가로 비율
        scale_y = frame_height / input_height  # 세로 비율
        # 원본 이미지 크기에 맞게 좌표 변환
        x1 = int(x1 * scale_x)
        y1 = int(y1 * scale_y)
        x2 = int(x2 * scale_x)
        y2 = int(y2 * scale_y)
        scale_w = int(w * scale_x)
        scale_h = int(h * scale_y)

        # 중심점 계산
        center_x = int(x1 + scale_w / 2)
        center_y = int(y1 + scale_h / 2)
        
        # 고유 객체 ID (ByteTrack에서 track_id 사용)
        object_id = target.track_id

        # 이전 위치와 비교하여 이동 방향 계산
        if object_id in object_tracks:
            prev_center_x, prev_center_y = object_tracks[object_id]

            # 디버깅 출력
            # print(f"객체 {object_id}: prev_center_y={prev_center_y}, center_y={center_y}, mid_y={mid_y}")

            # 위쪽 → 아래쪽 이동
            if prev_center_y < mid_y and center_y >= mid_y:
                count_num += 1
                print(f"객체 {object_id}가 위쪽에서 아래쪽으로 이동. 현재 인원수: {count_num}")

            # 아래쪽 → 위쪽 이동
            elif prev_center_y > mid_y and center_y <= mid_y:
                count_num -= 1
                print(f"객체 {object_id}가 아래쪽에서 위쪽으로 이동. 현재 인원수: {count_num}")

        # 현재 위치 저장
        object_tracks[object_id] = (center_x, center_y)
        
        # 바운딩 박스 그리기
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {object_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)  # 중심점 표시
    
    # 영상 중간 부분에 가로선 그리기
    cv2.line(frame, (0, mid_y), (frame_width, mid_y), (255, 0, 0), 1)  # (255, 0, 0)은 파란색, 두께는 1
    
    # 현재 카운팅 값을 화면에 표시
    cv2.putText(frame, f"Count: {count_num}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 프레임 보여주기
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


480 720
영상시작


c:\miniconda3\envs\p2\lib\site-packages\yolox\tracker\byte_tracker.py:182: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  dets_second = bboxes[inds_second]
c:\miniconda3\envs\p2\lib\site-packages\yolox\tracker\byte_tracker.py:185: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  scores_second = scores[inds_second]


객체 244가 아래쪽에서 위쪽으로 이동. 현재 인원수: -1
